## ML model for movie recommendation based on user input of film. Uses NearestNeighbor to determine the closest movie based on genres of the given movie input

In [84]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder

# Carregando os DataFrames a partir dos arquivos CSV
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

# Merge dos dataframes
merged_data = ratings.merge(movies, on='movieId')
result = merged_data[['userId', 'movieId', 'title', 'genres', 'rating']]

contagem_avaliacoes = result.groupby('userId').size().reset_index(name='qtd_avaliacoes')
contagem_avaliacoes = contagem_avaliacoes.loc[contagem_avaliacoes['qtd_avaliacoes'] > 30]
result = result[result['userId'].isin(contagem_avaliacoes['userId'])]

filmes_avaliacoes = result.groupby('movieId').size().reset_index(name='qtd_avaliacoes')
filmes_avaliacoes = filmes_avaliacoes.loc[filmes_avaliacoes['qtd_avaliacoes'] > 50]
filmes_avaliacoes = filmes_avaliacoes.merge(result[['movieId', 'title', 'genres']], on='movieId', how='left').drop_duplicates()
result = result[result['movieId'].isin(filmes_avaliacoes['movieId'])]

mlb = MultiLabelBinarizer()
genres_one_hot = mlb.fit_transform(result['genres'].str.split('|'))
genres_one_hot_df = pd.DataFrame(genres_one_hot, columns=mlb.classes_, index=result.index)

result_with_genres = pd.concat([result, genres_one_hot_df], axis=1)

result_pivot = result_with_genres.pivot_table(index=['movieId','title', 'genres'], values='rating', fill_value=0)
result_pivot.fillna(0.0, inplace=True)
filmes_sparse = csr_matrix(result_pivot)
# Criando e treinando o modelo preditivo
modelo = NearestNeighbors(algorithm = 'brute')
modelo.fit(filmes_sparse)
#print(result_pivot.iloc[0]) #ToyStory
distances, suggestions = modelo.kneighbors(result_pivot.iloc[0].values.reshape(1, -1))
for i in range(len(suggestions)):
    print(result_pivot.index[suggestions[i]])


MultiIndex([(   1,                           'Toy Story (1995)', ...),
            ( 924,               '2001: A Space Odyssey (1968)', ...),
            (1073, 'Willy Wonka & the Chocolate Factory (1971)', ...),
            (1240,                     'Terminator, The (1984)', ...),
            (1028,                        'Mary Poppins (1964)', ...)],
           names=['movieId', 'title', 'genres'])


## ML model for prediction of movie rating KNeighborsRegressor

In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors, KNeighborsRegressor
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import OneHotEncoder

movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

merged_data = ratings.merge(movies, on='movieId')
result = merged_data[['userId', 'movieId', 'title', 'genres', 'rating']]

contagem_avaliacoes = result.groupby('userId').size().reset_index(name='qtd_avaliacoes')
contagem_avaliacoes = contagem_avaliacoes.loc[contagem_avaliacoes['qtd_avaliacoes'] > 30]
result = result[result['userId'].isin(contagem_avaliacoes['userId'])]

filmes_avaliacoes = result.groupby('movieId').size().reset_index(name='qtd_avaliacoes')
filmes_avaliacoes = filmes_avaliacoes.loc[filmes_avaliacoes['qtd_avaliacoes'] > 50]
filmes_avaliacoes = filmes_avaliacoes.merge(result[['movieId', 'title', 'genres']], on='movieId', how='left').drop_duplicates()
result = result[result['movieId'].isin(filmes_avaliacoes['movieId'])]

mlb = MultiLabelBinarizer()
genres_one_hot = mlb.fit_transform(result['genres'].str.split('|'))
genres_one_hot_df = pd.DataFrame(genres_one_hot, columns=mlb.classes_, index=result.index)

result_with_genres = pd.concat([result, genres_one_hot_df], axis=1)

result_pivot = result_with_genres.pivot_table(index=['movieId', 'genres'], values='rating', fill_value=0)
result_pivot.fillna(0.0, inplace=True)

filmes_sparse = csr_matrix(result_pivot)
X_train, X_test, y_train, y_test = train_test_split(result_pivot.index, result_pivot.values, test_size=0.2, random_state=42)

# Converting the MultiIndex to a DataFrame
X_train_df = X_train.to_frame(index=False)
X_test_df = X_test.to_frame(index=False)

encoder = OneHotEncoder(sparse=False, handle_unknown='ignore')
X_train_encoded = encoder.fit_transform(X_train_df)
X_test_encoded = encoder.transform(X_test_df)
 
y_train_df = pd.DataFrame(y_train, columns=['rating'])

modelo = KNeighborsRegressor()
modelo.fit(X_train_encoded, y_train_df)
print(y_train_df)
# Make predictions on the test set
predictions = modelo.predict(X_test_encoded)

# Calculate RMSE and MAE
rmse = np.sqrt(mean_squared_error(y_test, predictions))
mae = mean_absolute_error(y_test, predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")

       rating
0    3.483607
1    3.518797
2    3.466019
3    3.703125
4    3.888889
..        ...
334  3.540000
335  3.306034
336  3.475806
337  3.711538
338  3.539474

[339 rows x 1 columns]
Root Mean Squared Error (RMSE): 0.38576959914593606
Mean Absolute Error (MAE): 0.3181010739912461


/home/ian/projetos/MACHINE_LEARNING/venv/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
